In [13]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

boston=load_boston()
dfX=pd.DataFrame(boston.data, columns=boston.feature_names)
dfy=pd.DataFrame(boston.target, columns=["MEDV"])
df=pd.concat([dfX,dfy], axis=1)

N=len(df) # 샘플 개수
ratio=0.7
np.random.seed(0)

idx_train=np.random.choice(np.arange(N), np.int(ratio*N), replace=False)
idx_test=list(set(np.arange(N))-set(idx_train))

df_train=df.iloc[idx_train]
df_test=df.iloc[idx_test]

<ipython-input-13-06e8615d514f>:14: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  idx_train=np.random.choice(np.arange(N), np.int(ratio*N), replace=False)


In [14]:
print(len(idx_train),len(idx_test))

354 152


In [15]:
import statsmodels.api as sm

model=sm.OLS.from_formula("MEDV ~ "+'+'.join(boston.feature_names), data=df_train)
result=model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.728
Model:                            OLS   Adj. R-squared:                  0.718
Method:                 Least Squares   F-statistic:                     70.06
Date:                Mon, 06 Dec 2021   Prob (F-statistic):           8.57e-88
Time:                        10:32:17   Log-Likelihood:                -1043.0
No. Observations:                 354   AIC:                             2114.
Df Residuals:                     340   BIC:                             2168.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     35.0719      5.932      5.913      0.0

In [16]:
pred=result.predict(df_test)
rss=((df_test.MEDV-pred)**2).sum()
tss=((df_test.MEDV-df_test.MEDV.mean())**2).sum()
rsquared=1-rss/tss
rsquared

0.7519796502601109

In [17]:
from sklearn.model_selection import train_test_split

df_train, df_test=train_test_split(df, test_size=0.3, random_state=0)
df_train.shape, df_test.shape

((354, 14), (152, 14))

In [18]:
dfX_train, dfX_test, dfy_train, dfy_test=train_test_split(dfX, dfy, test_size=0.3, random_state=0)
dfX_train.shape, dfy_train.shape, dfX_test.shape, dfy_test.shape

((354, 13), (354, 1), (152, 13), (152, 1))

In [19]:
from sklearn.model_selection import KFold

scores=np.zeros(5) # 0 다섯개 배열
cv=KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train=df.iloc[idx_train]
    df_test=df.iloc[idx_test]

    model=sm.OLS.from_formula("MEDV ~ "+'+'.join(boston.feature_names), data=df_train)
    result=model.fit()

    pred=result.predict(df_test)
    rss=((df_test.MEDV-pred)**2).sum()
    tss=((df_test.MEDV-df_test.MEDV.mean())**2).sum()
    rsquared=1-rss/tss

    scores[i]=rsquared
    print(f'학습용 R2={result.rsquared:.3f}, 검증용 R2={rsquared:.3f}')

학습용 R2=0.773, 검증용 R2=0.589
학습용 R2=0.729, 검증용 R2=0.778
학습용 R2=0.749, 검증용 R2=0.668
학습용 R2=0.757, 검증용 R2=0.668
학습용 R2=0.705, 검증용 R2=0.840


In [20]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

scores1=np.zeros(5) # 0 다섯개 배열
scores2=np.zeros(5)
scores3=np.zeros(5)
cv=KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train=df.iloc[idx_train]
    df_test=df.iloc[idx_test]

    model=sm.OLS.from_formula("MEDV ~ "+'+'.join(boston.feature_names), data=df_train)
    result=model.fit()

    pred=result.predict(df_test)
    rsquared=r2_score(df_test.MEDV, pred) # 모형의 설명력
    scores1[i]=rsquared
    mse=mean_squared_error(df_test.MEDV, pred) # 평균 제곱 오차(Mean Squared Error) : 오차 제곱 합계의 평균
    scores2[i]=mse
    mae=mean_absolute_error(df_test.MEDV, pred) # 평균 절대 오차(Mean Squared Error) : 오차 합계의 평균
    scores3[i]=mae

print(scores1)
print(scores2)
print(scores3)

[0.58922238 0.77799144 0.66791979 0.6680163  0.83953317]
[33.44898    18.65881615 21.23463289 29.22251557 16.57369039]
[3.84290922 3.38979394 3.07473854 3.6463452  3.03058651]
